# Create train set

In [1]:
from datasets import load_dataset

ds = load_dataset("shunk031/jsnli", 'with-filtering')

0000.parquet:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

with-filtering/validation/0000.parquet:   0%|          | 0.00/301k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/533005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3916 [00:00<?, ? examples/s]

In [2]:
rows = list(ds['train']) # + list(ds['validation'])
print(len(rows))

# 2 contradiction
# 0 entailment
rows = [r for r in rows if r['label'] != 1]
print(len(rows))

533005
355009


In [3]:
sents = set([r['premise'] for r in rows] + [r['hypothesis'] for r in rows])
sent2id = {s:i for i, s in enumerate(sents)}
id2sent = {i:s for s, i in sent2id.items()}
pairs_with_lb = [(set([sent2id[r['premise']], sent2id[r['hypothesis']]]), r['label']) for r in rows]

In [4]:
sent_lb_count = {}
for r in rows:
    idx = sent2id[r['premise']]
    if idx not in sent_lb_count:
        sent_lb_count[idx] = set()
    sent_lb_count[idx].add(r['label'])

    idx = sent2id[r['hypothesis']]
    if idx not in sent_lb_count:
        sent_lb_count[idx] = set()
    sent_lb_count[idx].add(r['label'])

lb_sent_count = {}
for ls in sent_lb_count.values():
    if str(ls) not in lb_sent_count:
        lb_sent_count[str(ls)] = 0
    lb_sent_count[str(ls)] += 1

lb_sent_count

{'{2}': 148876, '{0, 2}': 147476, '{0}': 132644}

In [ ]:
from collections import defaultdict

def create_triplets(pairs):
    shared_index_pairs = defaultdict(list)

    # Map each index to its corresponding pairs and labels
    for (idx_set, label) in pairs:
#         print(idx_set)
        if len(idx_set) == 1:
            continue
        idx1, idx2 = idx_set
        shared_index_pairs[idx1].append((idx2, label))
        shared_index_pairs[idx2].append((idx1, label))

    # Create triplets
    triplets = {}
    for idx, connections in shared_index_pairs.items():
        # Separate connections by label
        label_0_connections = {conn for conn, lbl in connections if lbl == 0}
        label_2_connections = {conn for conn, lbl in connections if lbl == 2}

        # Create all triplets with one 0-label connection and one 2-label connection
        for idx0 in label_0_connections:
            for idx2 in label_2_connections:
                # Form the triplet and add to the result set
                triplet = [idx, idx0, idx2]
                triplets[str(triplet)] = triplet

    # Return the list of unique triplets
    return list(triplets.values())

In [9]:
triplets = create_triplets(pairs_with_lb)

In [10]:
triplet_samples = [
    {
        'sent0': id2sent[t[0]],
        'sent1': id2sent[t[1]],
        'hard_neg': id2sent[t[2]],
    } for t in triplets
]

In [11]:
import pandas as pd

df = pd.DataFrame(triplet_samples)
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv("jsnli_for_simcse.csv", index=False)

In [12]:
df.duplicated(['sent0', 'sent1', 'hard_neg']).sum()

0

In [13]:
df.head()

,sent0,sent1,hard_neg
0,人々 の グループ が レース を して い ます 。,これ は レース に 参加 して いる 人々 の グループ です,左 の エスカレーター を 使用 して いる 人 も いれば 、 右 の 階段 を 使用 し...
1,赤 の 選手 が 防御 しよう と し ながら 、 青 の サッカー 選手 が ボール を ...,青 チーム の プレーヤー は 自分 の チーム が サッカー の ゴール を 確実に 得よ...,金 チーム は ボール を 保持 し 、 優しく 歌って い ます 。
2,２ 匹 の 犬 が 水 遊び を し ます 。,犬 は 濡れて いる,黒 犬 が 水しぶき 。
3,この クラフト の 苦痛 の 仕事 は 目 に 楽しい です 。,クラフト は 塗装 さ れて い ます 。,クラフト は 錆びて おり 、 むき出しです 。
4,女性 の 曲芸 師 が ステージ で 彼女 の ソロ パフォーマンス を 練習 し ます 。,人間 が 舞台 に い ます 。,女性 の 曲芸 師 が ベッド で 寝て い ます 。
